In [8]:
import torch
from torch import nn

##### Example 1

In [9]:
residual = torch.randn((5, 3, 64, 64))

In [10]:
from torch import nn

Write a `ShortcutProjection` to downsample `in_channels` of the residual to the target `in_channels`.

Hint: `kernel_size` is 1, `stride` follows argument

In [11]:
class ShortcutProjection(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels=in_channels, out_channels=out_channels,
            kernel_size=1, stride=stride
        )
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    
    def forward(self, x):
        return self.bn(self.conv(x))

In [12]:
residual.shape

torch.Size([5, 3, 64, 64])

Downsample `residual` to target `6` in_channels

In [13]:
shortcut = ShortcutProjection(in_channels=3, out_channels=6, stride=1)

In [14]:
shortcut(residual).shape

torch.Size([5, 6, 64, 64])

##### Example 2

In [11]:
residual = torch.randn(5, 3, 64, 64)

Hints:
- First convolution use `stride=1`, no padding
- Second convolution use `stride` as parameter, `padding=1`
- The third conv use `stride=1`, no padding

In [12]:
from torch import nn

Given `ShortcutProjection(in_channels, out_channels, stride)` use to downsample the residual.

Write `BottleneckResidualBlock` from scratch

In [13]:
class BottleneckResidualBlock(nn.Module):
    def __init__(self, in_channels, bottleneck_channels, out_channels, stride):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=bottleneck_channels,
            kernel_size=1, stride=1
        )
        self.bn1 = nn.BatchNorm2d(num_features=bottleneck_channels)
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv2d(
            in_channels=bottleneck_channels, out_channels=bottleneck_channels,
            kernel_size=3, stride=stride, padding=1
        )
        self.bn2 = nn.BatchNorm2d(num_features=bottleneck_channels)
        self.act2 = nn.ReLU()
        self.conv3 = nn.Conv2d(
            in_channels=bottleneck_channels, out_channels=out_channels,
            kernel_size=1, stride=1
        )
        self.bn3 = nn.BatchNorm2d(num_features=out_channels)
        if stride != 1 or in_channels != out_channels:
            self.shortcut = ShortcutProjection(
                in_channels=in_channels, out_channels=out_channels,
                stride=stride
            )
        else:
            self.shortcut = nn.Identity()
        
        self.act3 = nn.ReLU()
    
    def forward(self, x):
        residual = self.shortcut(x)
        out = self.act1(self.bn1(self.conv1(x)))
        out = self.act2(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
            
        return self.act3(out + residual)

In [14]:
residual.shape

torch.Size([5, 3, 64, 64])

In [15]:
bottleneck = BottleneckResidualBlock(in_channels=3, bottleneck_channels=4, out_channels=6, stride=2)

In [16]:
bottleneck(residual).shape

torch.Size([5, 6, 32, 32])

##### Example 3

Write ResNet-34 from scratch given `ResidualBlock(in_channels, out_channels, stride)`